# Final Capstone Project: Top of the cities

This notebook show how I using capstone project to help my friend to find a suitable place to open up a hotel.

## 1. Business problem

My colleage wants to open up a hotel in populated city in Vietnam. But he lacks of information about the current situation of existing hotels to detect where the best place to build a hotel is. This project aims at finding an appropriate city to open it.This project can be used by anyone who wishes to open a hotel or any other retail business in Vietnam.

## 2. Data requirements

The data needed for this project would be the name of the cities in Vietnam and their respective latitudes and longitudes, also the name, id , latitude and longitude of the respective venues in and around the cities of Vietnam. Such data are provided freely from this website https://simplemaps.com/data/vn-cities, where the data is available in the form of a csv file and also the name, id , latitude and longitude of the respective venues.

### Packages that need to be installed for working the data

* Geopy
* Geocoder
* Folium

Libraries needed for this project

* numpy
* pandas
* random
* requests
* matplotlib
* KMeans
* folium
* json_normalize
* Nominatim

First we must install necessary libraries:

In [1]:
!pip install geopy

In [2]:
! pip install geocoder

In [3]:
!pip install folium

Import necessary libraries:

In [4]:
import pandas as pd
import numpy as np
import random
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

Create dataframe of cities of Vietnam:

In [24]:
data_file= 'D://vn.csv'
df=pd.read_csv(data_file, sep=',')
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Ho Chi Minh City,10.8167,106.6333,Vietnam,VN,Hồ Chí Minh,admin,13312000.0,7431000.0
1,Hanoi,21.0245,105.8412,Vietnam,VN,Hà Nội,primary,7785000.0,7785000.0
2,Haiphong,20.8000,106.6667,Vietnam,VN,Hải Phòng,admin,2103500.0,2103500.0
3,Cần Thơ,10.0333,105.7833,Vietnam,VN,Cần Thơ,admin,1237300.0,1237300.0
4,Biên Hòa,10.9575,106.8426,Vietnam,VN,Đồng Nai,admin,1104000.0,1104000.0


In [28]:
df.shape

(64, 9)

## 3. Data pre-processing

Drop all row in the dataframe which have Nan value in population:

In [27]:
df.dropna(subset=["population"], axis=0, inplace=True) 

In [29]:
df.rename(columns = {'lat':'Latitude','lng':'Longitude','admin_name':'Region'},inplace=True)
df.head()

,city,Latitude,Longitude,country,iso2,Region,capital,population,population_proper
0,Ho Chi Minh City,10.8167,106.6333,Vietnam,VN,Hồ Chí Minh,admin,13312000.0,7431000.0
1,Hanoi,21.0245,105.8412,Vietnam,VN,Hà Nội,primary,7785000.0,7785000.0
2,Haiphong,20.8000,106.6667,Vietnam,VN,Hải Phòng,admin,2103500.0,2103500.0
3,Cần Thơ,10.0333,105.7833,Vietnam,VN,Cần Thơ,admin,1237300.0,1237300.0
4,Biên Hòa,10.9575,106.8426,Vietnam,VN,Đồng Nai,admin,1104000.0,1104000.0


Removing cities having population less than 10000:

In [30]:
length = df.shape[0]
for i in range(0,length):
    if (df['population'][i]<10000):
        df.drop([i],axis=0, inplace= True)
df.shape

(64, 9)

No cities removed from this step!

Showing the capital of Vietnam and the capital of its respective region


In [31]:
tempdf = df
tempdf = tempdf.groupby('capital')
tempdf.get_group('primary')

,city,Latitude,Longitude,country,iso2,Region,capital,population,population_proper
1,Hanoi,21.0245,105.8412,Vietnam,VN,Hà Nội,primary,7785000.0,7785000.0


Showing capitals of the respectve regions of Vietnam


In [32]:
tempdf.get_group('admin')

,city,Latitude,Longitude,country,iso2,Region,capital,population,population_proper
0,Ho Chi Minh City,10.8167,106.6333,Vietnam,VN,Hồ Chí Minh,admin,13312000.0,7431000.0
2,Haiphong,20.8000,106.6667,Vietnam,VN,Hải Phòng,admin,2103500.0,2103500.0
3,Cần Thơ,10.0333,105.7833,Vietnam,VN,Cần Thơ,admin,1237300.0,1237300.0
4,Biên Hòa,10.9575,106.8426,Vietnam,VN,Đồng Nai,admin,1104000.0,1104000.0
6,Bắc Ninh,21.1861,106.0763,Vietnam,VN,Bắc Ninh,admin,520000.0,520000.0
7,Hải Dương,20.9411,106.3331,Vietnam,VN,Hải Dương,admin,507469.0,507469.0
8,Vinh,18.6733,105.6922,Vietnam,VN,Nghệ An,admin,490000.0,490000.0
9,Huế,16.4667,107.5833,Vietnam,VN,Thừa Thiên-Huế,admin,455230.0,455230.0
10,Thanh Hóa,19.8075,105.7764,Vietnam,VN,Thanh Hóa,admin,393294.0,393294.0
11,Nha Trang,12.2500,109.1833,Vietnam,VN,Khánh Hòa,admin,392279.0,392279.0


Ploting the cities on the map of Vietnam using folium:

In [33]:
address = 'Vietnam'
geolocator = Nominatim(user_agent="vn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vietnam are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vietnam are 13.2904027, 108.4265113.


In [34]:
vn_map = folium.Map(location=[latitude, longitude], zoom_start=6)
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='white',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(vn_map)  
    
vn_map

Now using Foursquare to get information about venues around the cities:

In [35]:
CLIENT_ID = '2VTMKEILEXMUMNYJZ3TNOCZZDOLI1ME01L1SPYESIXFJE1EC'
CLIENT_SECRET = 'MK5QKWSCBGVWNAGTWZCDODETEFXAVUPI1F0ITN2AA5MTEYH1'
VERSION = '20180605'
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    vn_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    vn_venues.columns = ['city', 
                  'Lat', 
                  'Long', 
                  'Venue',
                  'Venue Id',       
                  'Venue Lat', 
                  'Venue Long', 
                  'Venue Category']
    
    return(vn_venues)

In [36]:
vn_venues = getNearbyVenues(names=df['city'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Ho Chi Minh City
Hanoi
Haiphong
Cần Thơ
Biên Hòa
Quảng Hà
Bắc Ninh
Hải Dương
Vinh
Huế
Thanh Hóa
Nha Trang
Nam Định
Buôn Ma Thuột
Thái Nguyên
Vũng Tàu
Cà Mau
Quy Nhơn
Sóc Trăng
Long Xuyên
Việt Trì
Thái Bình
Quảng Ngãi
Ấp Đa Lợi
Rạch Giá
Thủ Dầu Một
Tuy Hòa
Bạc Liêu
Sa Đéc
Phan Thiết
Sơn Tây
Phan Rang-Tháp Chàm
Hạ Long
Hà Tĩnh
Đồng Hới
Châu Đốc
Cẩm Phả
Cao Lãnh
Lạng Sơn
Cam Ranh
Pleiku
Tân An
Trà Vinh
Ninh Bình
Tây Ninh
Cam Ranh
Mỹ Tho
Hội An
Hòa Bình
Vĩnh Long
Vị Thanh
Yên Bái
Quảng Trị
Phú Quốc
Lào Cai
Bến Tre
Bắc Giang
Cao Bằng
Bình Long
Hà Giang
Tuyên Quang
Bắc Kạn
Sơn La
Đông Hà


In [37]:
vn_venues.head()

,city,Lat,Long,Venue,Venue Id,Venue Lat,Venue Long,Venue Category
0,Ho Chi Minh City,10.8167,106.6333,Celadon City,4d76e815de42721edbab472b,10.802461,106.618018,Park
1,Ho Chi Minh City,10.8167,106.6333,Seventeen Coffee,4f2d36f9e4b0d411babf803b,10.825413,106.629618,Lounge
2,Ho Chi Minh City,10.8167,106.6333,Sân Golf Tân Sơn Nhất,540e8af5498e77fa82948544,10.829959,106.649974,Golf Course
3,Ho Chi Minh City,10.8167,106.6333,AEON Supermarket,54901356498e6c9bcb3c9cde,10.801783,106.618443,Supermarket
4,Ho Chi Minh City,10.8167,106.6333,IBIS SAIGON AIRPORT Hotel,581c6d1d4e1d9a7c32e3053f,10.813010,106.666183,Hotel


In [38]:
# Checking number of venues returned for each city.
vn_venues.groupby('city').count()

,Lat,Long,Venue,Venue Id,Venue Lat,Venue Long,Venue Category
city,,,,,,,
Biên Hòa,20,20,20,20,20,20,20
Buôn Ma Thuột,9,9,9,9,9,9,9
Bình Long,1,1,1,1,1,1,1
Bạc Liêu,4,4,4,4,4,4,4
Bắc Giang,9,9,9,9,9,9,9
...,...,...,...,...,...,...,...
Vị Thanh,4,4,4,4,4,4,4
Yên Bái,5,5,5,5,5,5,5
Đông Hà,6,6,6,6,6,6,6


In [40]:
# Using One-hot coding approach
# one hot encoding
vn_onehot = pd.get_dummies(vn_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
vn_onehot['city'] = vn_venues['city'] 

# move city column to the first column
fixed_columns = [vn_onehot.columns[-1]] + list(vn_onehot.columns[:-1])
vn_onehot = vn_onehot[fixed_columns]

vn_onehot.head(5)

,city,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Village,Vineyard,Waterfall,Whisky Bar,Zoo,Zoo Exhibit
0,Ho Chi Minh City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ho Chi Minh City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ho Chi Minh City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ho Chi Minh City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ho Chi Minh City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
vn_group= vn_onehot.groupby('city').mean().reset_index()
vn_group.head(5)

,city,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Village,Vineyard,Waterfall,Whisky Bar,Zoo,Zoo Exhibit
0,Biên Hòa,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0
1,Buôn Ma Thuột,0.000000,0.111111,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Bình Long,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bạc Liêu,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0
4,Bắc Giang,0.111111,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10 # top 10 venues 

indicators = ['st', 'nd', 'rd']

# creating  columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# creating a new dataframe having city names and its top 10 venues
v_venues = pd.DataFrame(columns=columns)
v_venues['city'] = vn_group['city']

for ind in np.arange(vn_group.shape[0]):
    v_venues.iloc[ind, 1:] = return_most_common_venues(vn_group.iloc[ind, :], num_top_venues)

v_venues.head()  # top 10 venues dataframe

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Biên Hòa,Vietnamese Restaurant,Multiplex,Shopping Mall,Hotel,Noodle House,Japanese Restaurant,Café,Asian Restaurant,Train Station,Pizza Place
1,Buôn Ma Thuột,Café,Hotel,Airport,Airport Lounge,History Museum,Asian Restaurant,Dessert Shop,Film Studio,Fast Food Restaurant,Farm
2,Bình Long,Vietnamese Restaurant,Zoo Exhibit,Department Store,Film Studio,Fast Food Restaurant,Farm,Factory,Electronics Store,Eastern European Restaurant,Doner Restaurant
3,Bạc Liêu,Vietnamese Restaurant,Restaurant,Café,Asian Restaurant,Zoo Exhibit,Department Store,Fast Food Restaurant,Farm,Factory,Electronics Store
4,Bắc Giang,Afghan Restaurant,Cosmetics Shop,Soccer Field,Food Service,Bar,Trail,Mobile Phone Shop,Vietnamese Restaurant,Hotel,Coffee Shop


Apply Kmeans Clustering with k=4:

In [46]:
# set number of clusters
k = 4

# droping city column to get only top 10 venues columns
vn_clust = vn_group.drop('city', 1) 

# run k-means clustering
# fit kmean model with fin_clust dataframe
kmean = KMeans(n_clusters=k, random_state=0).fit(vn_clust)

# check cluster labels generated for each row in the dataframe
kmean.labels_[0:10]

array([0, 3, 2, 3, 0, 0, 0, 3, 0, 1])

In [47]:
# add clustering labels
v_venues.insert(0, 'Cluster Labels', kmean.labels_)

# copying original city dataframe to new dataframe
vndf = df

# merge city dataframe with city venues to add latitude/longitude for each city
vndf = vndf.join(v_venues.set_index('city'), on='city')

vndf.head() # check new datframe and new cluster label column

,city,Latitude,Longitude,country,iso2,Region,capital,population,population_proper,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ho Chi Minh City,10.8167,106.6333,Vietnam,VN,Hồ Chí Minh,admin,13312000.0,7431000.0,0,Vietnamese Restaurant,Hotel,Coffee Shop,Café,Noodle House,Japanese Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Park,Department Store
1,Hanoi,21.0245,105.8412,Vietnam,VN,Hà Nội,primary,7785000.0,7785000.0,0,Coffee Shop,Hotel,Vietnamese Restaurant,Noodle House,BBQ Joint,Multiplex,Pizza Place,Park,Steakhouse,Golf Course
2,Haiphong,20.8000,106.6667,Vietnam,VN,Hải Phòng,admin,2103500.0,2103500.0,0,Hotel,Café,Supermarket,Market,Multiplex,Electronics Store,Asian Restaurant,Lounge,Opera House,Ukrainian Restaurant
3,Cần Thơ,10.0333,105.7833,Vietnam,VN,Cần Thơ,admin,1237300.0,1237300.0,0,Hotel,Vietnamese Restaurant,Coffee Shop,Resort,Farm,Café,Multiplex,Floating Market,Museum,Market
4,Biên Hòa,10.9575,106.8426,Vietnam,VN,Đồng Nai,admin,1104000.0,1104000.0,0,Vietnamese Restaurant,Multiplex,Shopping Mall,Hotel,Noodle House,Japanese Restaurant,Café,Asian Restaurant,Train Station,Pizza Place


Plotting cities of Vietnam on the map of Vietnam in clusters:

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vndf['Latitude'], vndf['Longitude'], vndf['city'], vndf['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Results of Cluster:

### Final results

Show the city for each cluster:

In [49]:
vndf.loc[vndf['Cluster Labels'] == 0,vndf.columns[[0] + list(range(5, vndf.shape[1]))]]

,city,Region,capital,population,population_proper,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ho Chi Minh City,Hồ Chí Minh,admin,13312000.0,7431000.0,0,Vietnamese Restaurant,Hotel,Coffee Shop,Café,Noodle House,Japanese Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Park,Department Store
1,Hanoi,Hà Nội,primary,7785000.0,7785000.0,0,Coffee Shop,Hotel,Vietnamese Restaurant,Noodle House,BBQ Joint,Multiplex,Pizza Place,Park,Steakhouse,Golf Course
2,Haiphong,Hải Phòng,admin,2103500.0,2103500.0,0,Hotel,Café,Supermarket,Market,Multiplex,Electronics Store,Asian Restaurant,Lounge,Opera House,Ukrainian Restaurant
3,Cần Thơ,Cần Thơ,admin,1237300.0,1237300.0,0,Hotel,Vietnamese Restaurant,Coffee Shop,Resort,Farm,Café,Multiplex,Floating Market,Museum,Market
4,Biên Hòa,Đồng Nai,admin,1104000.0,1104000.0,0,Vietnamese Restaurant,Multiplex,Shopping Mall,Hotel,Noodle House,Japanese Restaurant,Café,Asian Restaurant,Train Station,Pizza Place
5,Quảng Hà,Quảng Nam,NaN,1000000.0,887069.0,0,Hotel,Resort,Coffee Shop,Vietnamese Restaurant,Restaurant,Spa,Beach,Café,Hotel Bar,BBQ Joint
6,Bắc Ninh,Bắc Ninh,admin,520000.0,520000.0,0,Restaurant,Hotel,Cosmetics Shop,Zoo Exhibit,Department Store,Fast Food Restaurant,Farm,Factory,Electronics Store,Eastern European Restaurant
7,Hải Dương,Hải Dương,admin,507469.0,507469.0,0,Hotel,Vietnamese Restaurant,Rest Area,Supermarket,Zoo Exhibit,Department Store,Fast Food Restaurant,Farm,Factory,Electronics Store
8,Vinh,Nghệ An,admin,490000.0,490000.0,0,Hotel,Park,Airport,Supermarket,Airport Lounge,Airport Service,Temple,Korean Restaurant,Electronics Store,Doner Restaurant
9,Huế,Thừa Thiên-Huế,admin,455230.0,455230.0,0,Hotel,Vietnamese Restaurant,Café,Historic Site,Bed & Breakfast,Vegetarian / Vegan Restaurant,Noodle House,Hotel Pool,Breakfast Spot,Monument / Landmark


In [50]:
vndf.loc[vndf['Cluster Labels'] == 1,vndf.columns[[0] + list(range(5, vndf.shape[1]))]]

,city,Region,capital,population,population_proper,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Thái Bình,Thái Bình,admin,268167.0,268167.0,1,Hotel,Bus Station,Pizza Place,Department Store,Fast Food Restaurant,Farm,Factory,Electronics Store,Eastern European Restaurant,Doner Restaurant
40,Pleiku,Gia Lai,admin,142900.0,114225.0,1,Hotel,Airport,Vietnamese Restaurant,Bus Station,Zoo Exhibit,Dessert Shop,Film Studio,Fast Food Restaurant,Farm,Factory
57,Cao Bằng,Cao Bằng,admin,41112.0,41112.0,1,Hotel,Pizza Place,Café,Zoo Exhibit,Department Store,Fast Food Restaurant,Farm,Factory,Electronics Store,Eastern European Restaurant
60,Tuyên Quang,Tuyên Quang,admin,36430.0,36430.0,1,Hotel,Asian Restaurant,Bus Station,Cocktail Bar,Coffee Shop,Film Studio,Fast Food Restaurant,Farm,Factory,Electronics Store


In [51]:
vndf.loc[vndf['Cluster Labels'] == 2,vndf.columns[[0] + list(range(5, vndf.shape[1]))]]

,city,Region,capital,population,population_proper,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Yên Bái,Yên Bái,admin,96540.0,96540.0,2,Vietnamese Restaurant,Hotpot Restaurant,Coffee Shop,Zoo Exhibit,Dessert Shop,Film Studio,Fast Food Restaurant,Farm,Factory,Electronics Store
58,Bình Long,Bình Phước,NaN,40279.0,40279.0,2,Vietnamese Restaurant,Zoo Exhibit,Department Store,Film Studio,Fast Food Restaurant,Farm,Factory,Electronics Store,Eastern European Restaurant,Doner Restaurant


In [52]:
vndf.loc[vndf['Cluster Labels'] == 3,vndf.columns[[0] + list(range(5, vndf.shape[1]))]]

,city,Region,capital,population,population_proper,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Thanh Hóa,Thanh Hóa,admin,393294.0,393294.0,3,Café,Hotel,Grocery Store,Bridge,Asian Restaurant,Train Station,Zoo Exhibit,Dessert Shop,Fast Food Restaurant,Farm
13,Buôn Ma Thuột,Đắk Lắk,admin,340000.0,340000.0,3,Café,Hotel,Airport,Airport Lounge,History Museum,Asian Restaurant,Dessert Shop,Film Studio,Fast Food Restaurant,Farm
14,Thái Nguyên,Thái Nguyên,admin,330000.0,330000.0,3,Café,Hotel,Resort,Coffee Shop,Restaurant,Fried Chicken Joint,Nature Preserve,Zoo Exhibit,Department Store,Factory
19,Long Xuyên,An Giang,admin,278658.0,278658.0,3,Shopping Mall,Boat or Ferry,Farm,Café,Bus Station,Fast Food Restaurant,Factory,Electronics Store,Eastern European Restaurant,Doner Restaurant
22,Quảng Ngãi,Quảng Ngãi,admin,260252.0,260252.0,3,Café,Coffee Shop,Vietnamese Restaurant,River,Bus Station,Zoo Exhibit,Dessert Shop,Fast Food Restaurant,Farm,Factory
23,Ấp Đa Lợi,Lâm Đồng,NaN,256019.0,131377.0,3,Café,Vietnamese Restaurant,Coffee Shop,Hotel,Waterfall,Lake,Snack Place,Restaurant,Golf Course,Sandwich Place
25,Thủ Dầu Một,Bình Dương,admin,244277.0,244277.0,3,Café,Shopping Mall,Multiplex,Vietnamese Restaurant,Cantonese Restaurant,Japanese Restaurant,Theme Park,Italian Restaurant,Cafeteria,Motel
27,Bạc Liêu,Bạc Liêu,admin,225000.0,150000.0,3,Vietnamese Restaurant,Restaurant,Café,Asian Restaurant,Zoo Exhibit,Department Store,Fast Food Restaurant,Farm,Factory,Electronics Store
36,Cẩm Phả,Quảng Ninh,minor,156000.0,156000.0,3,Harbor / Marina,Park,Café,Boat Rental,Korean Restaurant,Zoo Exhibit,Diner,Film Studio,Fast Food Restaurant,Farm
37,Cao Lãnh,Đồng Tháp,admin,149837.0,149837.0,3,Café,Department Store,Hotel,Flea Market,Vietnamese Restaurant,Electronics Store,River,Zoo Exhibit,Farm,Factory


In [54]:
#list of cities suitable for opening a hotels
suit_city = vndf.loc[vndf['Cluster Labels'] == 1,vndf.columns[[0] + list(range(5, vndf.shape[1]))]]
suit_list = suit_city['city'].values.tolist()
suit_list

['Thái Bình', 'Pleiku', 'Cao Bằng', 'Tuyên Quang']

## 4. Conclusion

From the cluster results, I can tell my colleage that he can think of opening his first hotel in one of 4 cities listed above: Thai Binh, Pleiku, Cao Bang, Tuyen Quang.